In [1]:
import numpy as np
import sys 
sys.path.append(r'../Python Script/')

from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy import *
from sympy.physics.mechanics import *
from sympy import *
import sympy

import torch
import sys 
sys.path.append(r'../../HLsearch/')
import HLsearch as HL
import matplotlib.pyplot as plt

In [2]:
import time

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

g = 9.81
m = 1
l = 1

def spherePend(t,y,Moment=0.0):
    from numpy import sin, cos
    theta, psi, theta_t, psi_t = y
    theta_2t = sin(theta)*cos(theta)*psi_t**2 - (g/l)*sin(theta)
    psi_2t = -2*theta_t*psi_t*cos(theta)/sin(theta)
    return theta_t,psi_t,theta_2t,psi_2t

In [3]:
#Saving Directory
rootdir = "../Spherical Pendulum/Data/"

num_sample = 100
create_data = False
training = True
save = False
noiselevel = 0

In [4]:
if(create_data):
    X, Xdot = [], []
    for i in range(num_sample):
        t = np.arange(0,5,0.01)
        theta = np.random.uniform(np.pi/3, np.pi/2)
        
        y0=np.array([theta, 0, 0, np.pi])
        x,xdot = generate_data(spherePend,t,y0)
        X.append(x)
        Xdot.append(xdot)
    X = np.vstack(X)
    Xdot = np.vstack(Xdot)
    if(save==True):
        np.save(rootdir + "X.npy", X)
        np.save(rootdir + "Xdot.npy",Xdot)
else:
    X = np.load(rootdir + "X.npy")
    Xdot = np.load(rootdir + "Xdot.npy")

In [5]:
#adding noise
mu, sigma = 0, noiselevel
noise = np.random.normal(mu, sigma, X.shape[0])
for i in range(X.shape[1]):
    X[:,i] = X[:,i]+noise
    Xdot[:,i] = Xdot[:,i]+noise

In [6]:
states_dim = 4
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

states are: (x0, x1, x0_t, x1_t)
states derivatives are:  (x0_t, x1_t, x0_tt, x1_tt)


In [7]:
#build function expression for the library in str
exprdummy = HL.buildFunctionExpressions(1,states_dim,states,use_sine=True)
polynom = exprdummy[1:4]
trig = [exprdummy[4], exprdummy[6]]
polynom = HL.buildFunctionExpressions(2,len(polynom),polynom)
trig = HL.buildFunctionExpressions(2, len(trig),trig)
product = []
for p in polynom:
    for t in trig:
        product.append(p + '*' + t)
expr = polynom + trig + product

In [8]:
#Creating library tensor
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

In [9]:
## separating known and unknown terms ##
expr = np.array(expr)
i1 = np.where(expr == 'x0_t**2')[0]
i2 = np.where(expr == 'x0_t**2*cos(x0)**2')[0]

idx = np.arange(0,len(expr))
idx = np.delete(idx,[i1,i2])
known_expr = expr[i1].tolist()
expr = np.delete(expr,[i1,i2])

#non-penalty index from prev knowledge
i3 = np.where(expr == 'cos(x0)')[0][0]
nonpenaltyidx=[i3]

expr = expr.tolist()

Zeta_ = Zeta[:,:,i1,:].clone().detach()
Eta_ = Eta[:,:,i1,:].clone().detach()
Delta_ = Delta[:,i1,:].clone().detach()

Zeta = Zeta[:,:,idx,:]
Eta = Eta[:,:,idx,:]
Delta = Delta[:,idx,:]

In [10]:
#Moving to Cuda
device = 'cuda:0'

Zeta = Zeta.to(device)
Eta = Eta.to(device)
Delta = Delta.to(device)

Zeta_ = Zeta_.to(device)
Eta_ = Eta_.to(device)
Delta_ = Delta_.to(device)

In [11]:
xi_L = torch.ones(len(expr), device=device).data.uniform_(-20,20)
prevxi_L = xi_L.clone().detach()
c = torch.ones(len(known_expr), device=device, requires_grad =False)

In [12]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [13]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

def proxL1norm(w_hat, alpha, nonpenaltyidx):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    for idx in nonpenaltyidx:
        w[idx] = w_hat[idx]
    return w

In [14]:
def training_loop(c, coef, prevcoef, RHS, LHS, xdot, bs, lr, lam, momentum=True):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]

    Zeta_, Eta_, Delta_ = LHS 
    Zeta, Eta, Delta = RHS

    if(torch.is_tensor(xdot)==False):
        xdot = torch.from_numpy(xdot).to(device).float()
    
    v = coef.clone().detach().requires_grad_(True)
    prev = v
    
    for i in range(tl//bs):
                
        #computing acceleration with momentum
        if(momentum==True):
            vhat = (v + ((i-1)/(i+2))*(v - prev)).clone().detach().requires_grad_(True)
        else:
            vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
   
        prev = v

        #Computing loss
        zeta = Zeta[:,:,:,i*bs:(i+1)*bs]
        eta = Eta[:,:,:,i*bs:(i+1)*bs]
        delta = Delta[:,:,i*bs:(i+1)*bs]

        zeta_ = Zeta_[:,:,:,i*bs:(i+1)*bs]
        eta_ = Eta_[:,:,:,i*bs:(i+1)*bs]
        delta_ = Delta_[:,:,i*bs:(i+1)*bs]
        
        x_t = xdot[i*bs:(i+1)*bs,:]

        #forward
        pred = -ELforward(vhat,zeta,eta,delta,x_t,device)
        targ = ELforward(c,zeta_,eta_,delta_,x_t,device)
        #targ = torque[:,i*bs:(i+1)*bs]
        
        lossval = loss(pred, targ)
        
        #Backpropagation
        lossval.backward()

        with torch.no_grad():
            v = vhat - lr*vhat.grad
            v = (proxL1norm(v,lr*lam,nonpenaltyidx))
            
            # Manually zero the gradients after updating weights
            vhat.grad = None
        
        
    
        
        loss_list.append(lossval.item())
    print("Average loss : " , torch.tensor(loss_list).mean().item())
    return v, prevcoef, torch.tensor(loss_list).mean().item()

In [15]:
Epoch = 100
i = 0
lr = 5e-6
lam = 5
temp = 1000
RHS = [Zeta, Eta, Delta]
LHS = [Zeta_, Eta_, Delta_]
while(i<=Epoch):
    print("\n")
    print("Stage 1")
    print("Epoch "+str(i) + "/" + str(Epoch))
    print("Learning rate : ", lr)
    xi_L, prevxi_L, lossitem= training_loop(c,xi_L,prevxi_L,RHS,LHS,Xdot,128,lr=lr,lam=lam)
    temp = lossitem
    i+=1



Stage 1
Epoch 0/100
Learning rate :  5e-06
Average loss :  76530.953125


Stage 1
Epoch 1/100
Learning rate :  5e-06
Average loss :  2939.46484375


Stage 1
Epoch 2/100
Learning rate :  5e-06
Average loss :  865.51513671875


Stage 1
Epoch 3/100
Learning rate :  5e-06
Average loss :  393.398681640625


Stage 1
Epoch 4/100
Learning rate :  5e-06
Average loss :  164.3717041015625


Stage 1
Epoch 5/100
Learning rate :  5e-06
Average loss :  94.33831024169922


Stage 1
Epoch 6/100
Learning rate :  5e-06
Average loss :  68.93724822998047


Stage 1
Epoch 7/100
Learning rate :  5e-06
Average loss :  60.98732376098633


Stage 1
Epoch 8/100
Learning rate :  5e-06
Average loss :  55.10904312133789


Stage 1
Epoch 9/100
Learning rate :  5e-06
Average loss :  49.97760009765625


Stage 1
Epoch 10/100
Learning rate :  5e-06
Average loss :  46.42514419555664


Stage 1
Epoch 11/100
Learning rate :  5e-06
Average loss :  42.0601806640625


Stage 1
Epoch 12/100
Learning rate :  5e-06
Average loss :  3

In [16]:
## Thresholding
threshold = 1e-2
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
L = HL.generateExpression(xi_Lcpu,expr,threshold=1e-3)
print(simplify(L))

0.96*x1_t**2*sin(x0)**2 + 18.93*cos(x0)


In [17]:
## Next round selection ##
for stage in range(3):

    #Redefine computation after thresholding
    expr.append(known_expr[0])
    Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

    expr = np.array(expr)
    i1 = np.where(expr == 'x0_t**2')[0]
    idx = np.arange(0,len(expr))
    idx = np.delete(idx,i1)
    known_expr = expr[i1].tolist()
    expr = np.delete(expr,i1)

    i4 = np.where(expr == 'cos(x0)')[0][0]
    nonpenaltyidx = [i4]

    expr = expr.tolist()

    Zeta_ = Zeta[:,:,i1,:].clone().detach()
    Eta_ = Eta[:,:,i1,:].clone().detach()
    Delta_ = Delta[:,i1,:].clone().detach()

    Zeta = Zeta[:,:,idx,:]
    Eta = Eta[:,:,idx,:]
    Delta = Delta[:,idx,:]

    Zeta = Zeta.to(device)
    Eta = Eta.to(device)
    Delta = Delta.to(device)
    Zeta_ = Zeta_.to(device)
    Eta_ = Eta_.to(device)
    Delta_ = Delta_.to(device)

    Epoch = 100
    i = 0
    lr += lr
    lam = 0.5
    temp = 1000
    RHS = [Zeta, Eta, Delta]
    LHS = [Zeta_, Eta_, Delta_]
    while(i<=Epoch):
        print("\n")
        print("Stage " + str(stage+2))
        print("Epoch "+str(i) + "/" + str(Epoch))
        xi_L, prevxi_L, lossitem= training_loop(c, xi_L,prevxi_L,RHS,LHS,Xdot,128,lr=lr,lam=lam)
        temp = lossitem
        if(temp < 1e-3):
            break
        i+=1
    
    ## Thresholding
    threshold = 1e-1
    surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    expr = np.array(expr)[surv_index].tolist()

    xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
    prevxi_L = xi_L.clone().detach()

    ## obtaining analytical model
    xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
    L = HL.generateExpression(xi_Lcpu,expr,threshold=1e-3)
    print("Result stage " + str(stage+2) + ":" , simplify(L))



Stage 2
Epoch 0/100
Average loss :  0.09015023708343506


Stage 2
Epoch 1/100
Average loss :  0.07852653414011002


Stage 2
Epoch 2/100
Average loss :  0.07212797552347183


Stage 2
Epoch 3/100
Average loss :  0.06582719087600708


Stage 2
Epoch 4/100
Average loss :  0.06072680279612541


Stage 2
Epoch 5/100
Average loss :  0.05556623637676239


Stage 2
Epoch 6/100
Average loss :  0.05014198645949364


Stage 2
Epoch 7/100
Average loss :  0.04613960534334183


Stage 2
Epoch 8/100
Average loss :  0.041373129934072495


Stage 2
Epoch 9/100
Average loss :  0.037552423775196075


Stage 2
Epoch 10/100
Average loss :  0.03345894068479538


Stage 2
Epoch 11/100
Average loss :  0.029747478663921356


Stage 2
Epoch 12/100
Average loss :  0.026519393548369408


Stage 2
Epoch 13/100
Average loss :  0.02279512956738472


Stage 2
Epoch 14/100
Average loss :  0.020229849964380264


Stage 2
Epoch 15/100
Average loss :  0.0171426422894001


Stage 2
Epoch 16/100
Average loss :  0.01488919835537672


S

Result stage 4: 1.0*x1_t**2*sin(x0)**2 + 19.56*cos(x0)


In [ ]:
## Adding known terms
L = str(simplify(L)) + " + " + known_expr[0]
print("\m")
print("Obtained Lagrangian: ", L)

expr = expr + known_expr
xi_L = torch.cat((xi_L, c))
mask = torch.ones(len(xi_L),device=device)

In [ ]:
if(save==True):
    #Saving Equation in string
    text_file = open(rootdir + "lagrangian_" + str(noiselevel)+ "_noise.txt", "w")
    text_file.write(L)
    text_file.close()